In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset, Dataset

# Data Processing

In [ ]:
# Define the dataset name
counselling_dataset = "nbertagnolli/counsel-chat"

# Load the training dataset
dataset = load_dataset(counselling_dataset, split="train")
formatted_data = [
    f"<s>[INST] {data['questionText']} [/INST] {data['answerText']} </s>" for data in dataset
]
formatted_dataset = Dataset.from_dict({"text": formatted_data})
print(formatted_dataset[1])

Repo card metadata block was not found. Setting CardData to empty.


{'text': '<s>[INST] I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling? [/INST] I\'ve never heard of someone having "too many issues" for therapy to be effective. A competent therapist will assist you in identifying the root causes of your problems and treat those first. If the underlying issues are addressed, your various symptoms should improve. For example, a history of sexual trauma can cause sleep disturbances, depression, anxiety, and low self-worth. I would start by addressing the underlying trauma using EMDR Therapy. EMDR allows the client to process unresolved trauma and to address negative core beliefs that develop in response to the trauma. By addressing t

## Reforming data for Llama2

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"
# Fine-tuned model
new_model = "llama-chat-mental_health_companion"

# 4 Bit Quantization

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Loading Llama Model

In [59]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

# Loading Tokenizer

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Trainning Parameters

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Model Finetuning

In [ ]:
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=1024
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params,
    sft_config=sft_config,
    packing=False,
)


In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

# Evaluation

In [ ]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))


# Trying out my model

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "I am depressed, what advice you can give"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])